# Reporting Basics - 1

To construct necessary reporting tables, we will need to use certain SAS tools to build reports. We will need 
* PROC FREQ
* PROC SUMMARY
* PROC REPORT
* PROC TEMPLATE
* PROC FORMAT

## PROC FREQ

The FREQ procedure produces one-way to n-way frequency and contingency (crosstabulation) tables.
* It calculates the number of observations and the percentages identified by a variable and put it in a operatable table. 

In [ ]:
proc sort data = sashelp.cars out = x; /* <-- Sort and load the data */
	by origin make model;
run;

data x;
	set x;
	id + 1; /* <-- Generate an ID */
run;

proc sort; /* <-- Note that I don't have the DATA = statement. By default, SAS will use the most recent data. *//* You can do it but I don't recommend. */
	by origin;
run;

proc freq noprint;  
	by origin; /* <-- Execute PROC FREQ for each origin. */ 
	table type / out = out_1; /* <-- The variable I want to tabulate is "type". I put the results in a table called "out_1". */
run;

Let's subset the data so that it is more managable. 
* Keep only the Acura and Mercedes-Benz cars.
* keep only SUVs and Sedans. 

In [ ]:
/* where or if (make = "Acura" or make = "Mercedes-Benz") and (type = "SUV" or type = "Sedan"); */

After we have filtered the data, we can re-do the frequency (proc freq). 

In [ ]:
proc sort data = sashelp.cars (where = ((make = "Acura" or make = "Mercedes-Benz") and (type = "SUV" or type = "Sedan"))) out = x; /* <-- Sort and load the data */
	by origin make model;
run;

data x;
	set x;
	id + 1; /* <-- Generate an ID */
run;

proc sort; /* <-- Note that I don't have the DATA = statement. By default, SAS will use the most recent data. *//* You can do it but I don't recommend. */
	by origin;
run;

proc freq noprint;  
	by origin; /* <-- Execute PROC FREQ for each origin. */ 
	table type / out = out_1; /* <-- The variable I want to tabulate is "type". I put the results in a table called "out_1". */
run;

Let's make the outcome into a professional looking table. 

Let's say we want this:

In [ ]:
/*
                    Acura       Mercedes-Benz       Overall
Type [n(%)]
    SUV            xx(xx.x)       xx(xx.x)          xx(xx.x)
    Sedan          xx(xx.x)       xx(xx.x)          xx(xx.x)
*/

The data out_1 should have all the required information. There are a couple special things about this table compared to a normal SAS table.
* There is an overall column to calculate statistics. In the previous results table, we only have by group results.
* n and % are reported next to each other in one column.
* There is an empty row with Type [n(%)].
* The rows begin wih SUC and Sedan and they are indented. 
* Column titles.

If we can handle these, we should be able to create this table. The entire code is below.

In [ ]:
data x;
	set sashelp.cars (where = ((make = "Acura" or make = "Mercedes-Benz") and (type = "SUV" or type = "Sedan")));
	id + 1; /* <-- Generate an ID */
    if make = "Acura" then makeid = 0;
    else if make = "Mercedes-Benz" then makeid = 1;
    output;
    makeid =2;
    output;
run;

proc sort; 
	by makeid;
run;

proc freq noprint;  
	by makeid; /* <-- Execute PROC FREQ for each make. */ 
	table type / out = out_1; /* <-- The variable I want to tabulate is "type". */
run;

data out_2;
	set out_1;
	_n = put(count,5.0)||"("||put(percent,5.1)||")";
	drop count percent;
run;


proc sort data = out_2;
	by type;
run;

proc transpose data = out_2 out = out_3 prefix = make;
	by type;
	id makeid;
	var _n;
run;

data out_3;
	set out_3 (drop = _Name_);
	if type = "SUV" then row_name = "    SUV";
	else if type = "Sedan" then row_name = "    Sedan";
run;

data z;
	row_name = "Type [n(%)]";
run;

data out_3;
	set z out_3;
run;

## PROC SUMMARY

In [ ]:
data x;
	set sashelp.cars (where = ((make = "Acura" or make = "Mercedes-Benz") and (type = "SUV" or type = "Sedan")));
	id + 1; /* <-- Generate an ID */
    if make = "Acura" then makeid = 0;
    else if make = "Mercedes-Benz" then makeid = 1;
    output;
    makeid =2;
    output;
run;

proc sort; 
	by makeid;
run;

proc summary;
    by makeid;
    var msrp;
    output out = out_1 N = _N mean = _mean std = _std median = _median min = _min max = _max;
run;

I want the following this time. Can you tell me what to do? Let's think about how we can figure it out. 

In the PROC FREQ example, all we did was turning PROC FREQ output into something that's consistent with the reporting format. We need to first learn how the output of PROC SUMMARY looks and then think about how to turn it into something consistent with the format. 

In [ ]:
/*
                    Acura       Mercedes-Benz       Overall
MSRP ($)
    N                xx             xx                xx
    Mean(SD)     xx.x(xx.xx)     xx.x(xx.xx)       xx.x(xx.xx)
    Median          xx.x           xx.x              xx.x
    Min, Max       xx, xx         xx, xx            xx, xx
*/

In [ ]:
/* Try to figure it out! */